# Shape of Molecules #

In this notebook we provide an innovative pipeline that makes it possible to find interesting and meaningful structural features by exploiting the package $\href{https://giotto.ai/}{giotto learn}$.

In [ ]:
#Import statements
import warnings; warnings.simplefilter('ignore')
import random 


import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from giotto.graphs.create_clique_complex import CreateCliqueComplex, CreateBoundaryMatrices, CreateLaplacianMatrices    
from giotto.graphs.heat_diffusion import HeatDiffusion
from giotto.graphs.graph_entropy import GraphEntropy

from molecules import mol_to_nx, compute_node_edge_entropy, bonds_type, graph_to_points, bonds_type_to_edge
from plotting import plot_entropies, plot_network_diffusion

from rdkit import Chem 
from rdkit.Chem import Draw


## Import and Convert data to networkx Graph ##

Import molecules dataset and convert them: $\textit{smiles}$ --> $\textit{rdkit.Chem.rdchem.Mol}$ --> $\textit{networkx.graph}$.

In [ ]:
#Import and convert data
df = pd.read_csv('hiv.csv')
df['g_mol'] = df['smiles'].apply(lambda x: Chem.MolFromSmiles(x))
df.drop("smiles", axis=1, inplace=True)
g_mol = [mol_to_nx(df['g_mol'][i]) for i in range(df.shape[0]) if i != 559 and i!= 8097 ]

## Plot one molecule ##

We plot here one example of a molecule: in the first plot it is represented as a $\textit{Mol}$ object of the $\textit{rdkit}$ module; in the second one the molecule is plotted as a graph by using the $\textit{networkx}$ package. 

In [ ]:
#Choose one random molecule
g = g_mol[0]
mol = [df['g_mol'][0]]
plt.figure(figsize=(6,3))
nx.draw(g, pos=nx.spring_layout(g, iterations=1000))
Draw.MolsToGridImage(mol, molsPerRow=5, useSVG=False)


## Create Embeddings for all atoms and bonds in the dataset ##

Here the embeddings for all atoms and bonds presented in the dataset are computed. In particular it is possible to choose different hyperparameters: the number of points in time at which the heat diffusions have to be sampled and the last sample instant. In this example the different samples are linearly spaced in time but of course it possible to choose them differently. 

In [ ]:
# Hyperparameters
taus_n = np.linspace(0,2,30)
taus_e = np.linspace(0,2,30)

In [ ]:
# Embedding
embeds = [compute_node_edge_entropy(x,i, taus_n, taus_e) for i,x in enumerate(g_mol) ]

## Save and Load Embeddings ##

This part of the notebook let the users save and reload the found embeddings for all molecules and embedding. It can be used to save time when trying different experiments.

In [ ]:
#Save all the Embeddings 
import pickle 
pickle.dump(embeds, open("hiv_embeds.pickle", "wb"))

In [ ]:
#Load it and look at the dimensions 
import pickle
embeds = pickle.load(open("hiv_embeds_40.pickle", 'rb'))
print("Dataset loaded...")

# Adding Bonds information #

We now add to each node chemical informatino about the type of bonds they make with neighbor nodes.

In [ ]:
bonds_type(g_mol)
bonds_type_to_edge(g_mol)

#Create a list with all nodes 
freq_type_bonds = list()
for g in g_mol:
    freq_type_bonds.extend(list(nx.get_node_attributes(g, 'bonds_one_hot').values()))
    
freq_type_bonds_edge = list()
for g in g_mol:
    freq_type_bonds_edge.extend(list(nx.get_edge_attributes(g, 'bonds_one_hot').values()))

In [ ]:
freq_type_bonds = np.array(freq_type_bonds)
freq_type_bonds_edge = np.array(freq_type_bonds_edge)
print("Total number of atoms in the dataset: {}".format(freq_type_bonds.shape[0]))
print("Total number of bonds in the dataset: {}".format(freq_type_bonds_edge.shape[0]))

# Universal Node Embeddding #

In [ ]:
universal_nodes = list()
for i in range(len(embeds)):
    universal_nodes.extend(np.split(embeds[i][0][1:,:].T, embeds[i][0][0,:].shape[0]))
universal_nodes = np.squeeze(np.array(universal_nodes))

In [ ]:
uni_frq_nodes = [np.hstack([universal_nodes[x,:], freq_type_bonds[x,:]]) for x in range(universal_nodes.shape[0])]
uni_frq_nodes = np.array(uni_frq_nodes)

In [ ]:
mol_to_atom = graph_to_points(g_mol, 0)

In [ ]:
kmeans_n = KMeans(n_clusters=10)
universal_class_nodes = kmeans_n.fit_transform(uni_frq_nodes)
 
centroids_n = kmeans_n.cluster_centers_

# Universal Edge Emebdding #

In [ ]:
universal_edges = list()
for i in range(len(embeds)):
    universal_edges.extend(np.split(embeds[i][1][1:,:].T, embeds[i][1][0,:].shape[0]))
universal_edges = np.squeeze(np.array(universal_edges))

In [ ]:
uni_frq_edges = [np.hstack([universal_edges[x,:], freq_type_bonds_edge[x,:]]) for x in range(universal_edges.shape[0])]
uni_frq_edges = np.array(uni_frq_edges)

In [ ]:
mol_to_bonds = graph_to_points(g_mol, 1)

In [ ]:
kmeans_e = KMeans(n_clusters=10)
universal_class_edge = kmeans_e.fit_transform(uni_frq_edges)

centroids_e = kmeans_e.cluster_centers_

# Data with universal embedding #


### Nodes ###

In [ ]:
"""
one_hot_encoded = np.zeros((universal_nodes.shape[0], 10))
for x in range(len(universal_points)):
    one_hot_encoded[x][universal_class[x]] = 1
"""

In [ ]:
#Soft Encoded
# soft_encoded_node = np.zeros((uni_frq.shape[0], 10))
soft_encoded_node = [[ np.exp( - (np.linalg.norm(uni_frq_nodes[x]- centroids_n[c], 2) ** 2) / 2) for c in range(10)] for x in range(uni_frq_nodes.shape[0])]
soft_encoded_node = np.array(soft_encoded_node)

In [ ]:
#Create data for each graph
x_data_node = [ np.sum([soft_encoded_node[n] for n in mol_to_atom[i]], axis=0) for i in range(len(g_mol))]
x_data_node = np.array(x_data_node)
print("Check shape of x_data_node: {}".format(x_data_node.shape))

### Edges ###

In [ ]:
"""
one_hot_encoded_edge = np.zeros((universal_class_edge.shape[0], 10))
for x in range(len(universal_edge)):
    one_hot_encoded_edge[x][universal_class_edge[x]] = 1
    """

In [ ]:
#Soft Encoded
soft_encoded_edge = [[ np.exp( - (np.linalg.norm(uni_frq_edges[x]- centroids_e[c], 2) ** 2) / 2) for c in range(10)] for x in range(universal_edges.shape[0])]
soft_encoded_edge = np.array(soft_encoded_edge)

In [ ]:
x_data_edge = [ np.sum([soft_encoded_edge[n] for n in mol_to_bonds[i]], axis=0)  for i in range(len(g_mol))]
x_data_edge = np.array(x_data_edge)
print("Check shape of x_data_edge: {}".format(x_data_edge.shape))

In [ ]:
x_data = np.hstack([x_data_node, x_data_edge])
x_data -= np.mean(x_data, axis=0)
x_data /= (np.max(x_data, axis=0) - np.min(x_data, axis=0))

# Classification Model #

In [ ]:
#Prepare y_data
y_data = [df['HIV_active'][i] for i in range(df.shape[0]) if i != 8079 and i != 559]
y_data = np.array(y_data)
y_data.shape

In [ ]:
import random

f = np.arange(41911)
random.Random(10).shuffle(f)
train = f[:30000]

In [ ]:
np.random.shuffle(train)

i_train = train[:36000]
i_val = train[30000:36000]
i_test = f[36000:]

x_train = x_data[i_train, :]
x_val = x_data[i_val, :] 

y_train = y_data[i_train]
y_val = y_data[i_val]

x_test = np.array([np.array(x_data[i,:]) for i in f[36000:]])
y_test = np.array([y_data[i] for i in f[36000:]])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras import optimizers

from sklearn.metrics import roc_auc_score

In [ ]:
## # define the keras model
model = Sequential()

model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(BatchNormalization())

model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

In [ ]:
adam = optimizers.Adam(lr=0.001)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=100,validation_data=(x_val, y_val), batch_size=128)

In [ ]:
# evaluate the keras model
pe, accuracy = model.evaluate(x_val, y_val)
print('Accuracy: %.2f' % (accuracy*100))

p_train = model.predict(x_train)
p_val = model.predict(x_val)
p_test = model.predict(x_test)

print(" Train AUC-ROC : {}".format(roc_auc_score(y_train, p_train)))

print(" Valid AUC-ROC : {}".format(roc_auc_score(y_val, p_val)))

print(" Test AUC-ROC : {}".format(roc_auc_score(y_test, p_test)))
